## Системы управления базами данных MongoDB 

В данном ноутбуке реализован парсинг основной информации по вакансиям с сайтов hh.ru и superjob.ru. Пользователь может передать через input ключевые слова для поиска, который сохраняется в поле query, и регион  (поиск работает для 12-ти крупных российских городов). Приложение анализирует все доступные страницы поиска.

Найденные вакансии сохраняются в коллекцию MongoDB-базы vacancies, содержащую следующие ключи:

- Запрос поиска
- Регион поиска
- ID вакансии (уникальный ключ, по которому совершается проверка наличия данной вакансии в коллекции)
- Должность
- Зарплата (отдельно минимум, максимум и валюта)
- Работодатель
- Cсылка на вакансию
- Источник вакансии
- Описание вакансии

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from pymongo import MongoClient
from pprint import pprint

In [2]:
client = MongoClient('localhost', 27017)
db = client['jobs_parser_db']

In [3]:
vacancies = db.vacancies

In [4]:
# словари для трансформации региона поиска
hh_area_dict = {
    'москва': '1',
    'санкт-петербург': '2',
    'екатеринбург': '3',
    'новосибирск': '4',
    'нижний новгород': '66',
    'казань': '88',
    'воронеж': '26',
    'волгоград': '24',
    'ростов-на-дону': '76',
    'краснодар': '53',
    'уфа': '99', 
    'хабаровск': '102'
}

sj_area_dict = {
    'москва': 'москва',
    'санкт-петербург': 'spb',
    'екатеринбург': 'ekaterinburg',
    'новосибирск': 'nsk',
    'нижний новгород': 'nn',
    'казань': 'kazan',
    'воронеж': 'voronezh',
    'волгоград': 'volgograd',
    'ростов-на-дону': 'rnd',
    'краснодар': 'krasnodar',
    'уфа': 'ufa', 
    'хабаровск': 'habarovsk'
}

In [5]:
# параметры поиска
query = input('Введите запрос: ').lower()
area = input('Введите город для поиска: ').lower()

Введите запрос: machine learning
Введите город для поиска: москва


In [6]:
# преобразовываем input в понятный парсеру
def hh_area_transform(area):
    if area in hh_area_dict:
        return hh_area_dict[area]
    else:
        print('Неизвестный регион')
    
def sj_area_transform(area):
    if area in sj_area_dict:
        return sj_area_dict[area]
    else:
        print('Неизвестный регион')
        
hh_area = hh_area_transform(area)
sj_area = sj_area_transform(area)

In [7]:
hh = 'https://hh.ru/'
sj = 'https://www.superjob.ru'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:74.0) Gecko/20100101 Firefox/74.0'}

# задаем счетчики страниц - у hh - c 0, у sj - c 1
hh_page_count = range(100)
sj_page_count = range(1,100)

# создаем паттерны для извлечения id вакансии
hh_id_pattern = re.compile(r'https://[^/]+/vacancy/(\d+)')
sj_id_pattern = re.compile(r'https://www.superjob.ru/vakansii/[\w+\-\w+]+\-(\d+)')

### hh.ru parsing

In [8]:
def get_title_hh(vacancy):
    return vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-title'})[0].getText()


def get_url_hh(vacancy):
    return vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-title'})[0]['href']
    

def get_id_hh(vacancy):
    return f'hh_{hh_id_pattern.findall(get_url_hh(vacancy))[0]}'


def get_employer_hh(vacancy):
    employer_el = vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
    # случай, когда работодатель указан описательно (нет ссылки на компанию)
    if employer_el:
        return employer_el[0].getText()
    else:
        return vacancy.findChildren('div', {'class': 'vacancy-serp-item__meta-info'})[0].getText() 

    
def get_salary_hh(vacancy):
    #  зарплата
    min_salary, max_salary, currency = None, None, None

    salary_el = vacancy.findChildren('span', {'data-qa': "vacancy-serp__vacancy-compensation"})

    if not salary_el:
        return min_salary, max_salary, currency
    
    # разбиваем по тире
    split_range = re.split('-', salary_el[0].getText())
    if len(split_range) == 2:
        min_salary = int(''.join(re.findall('\d+', split_range[0])))
        items = re.findall('\w+', split_range[1])
        max_salary = int(''.join(items[:-1]))
        currency = items[-1] 
    else:
        items = re.findall('\w+', split_range[0])
        if items[0] == 'от':
            min_salary = int(''.join(items[1:-1]))
        else:
            max_salary = int(''.join(items[1:-1]))         
        currency = items[-1]

    return min_salary, max_salary, currency


def get_description_hh(vacancy):
    desc1 = vacancy.findChildren('div', {'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'})[0].getText()
    desc2 = vacancy.findChildren('div', {'data-qa': 'vacancy-serp__vacancy_snippet_requirement'})[0].getText()
    return desc1 + ' ' + desc2


In [9]:
for i in hh_page_count:
    hh_link = f'{hh}search/vacancy?area={hh_area}&st=searchVacancy&text={query}&page={i}'
    hh_page = requests.get(hh_link, headers=headers)
    
    if not hh_page.ok:
        break
    
    # формируем список элементов страницы, содержащих вакансии
    hh_soup = bs(hh_page.text,'lxml')
    vacancies_block = hh_soup.find_all('div', {'class': 'vacancy-serp'})[0]
    vacancies_list = vacancies_block.find_all('div', {'class': 'vacancy-serp-item'})
    
    # определяем условие выхода из цикла (вакансий больше нет)
    if len(vacancies_list) == 0:
        break
    
    # итерируемся по списку, содержащем вакансии
    for vacancy in vacancies_list:
        salary = get_salary_hh(vacancy)

        vacancy_data = {
            'query': query,
            'area': area,
            'url': get_url_hh(vacancy),
            'id': get_id_hh(vacancy),
            'title': get_title_hh(vacancy),
            'employer': get_employer_hh(vacancy),
            'min_salary': salary[0],
            'max_salary': salary[1],
            'currency': salary[2],
            'source': hh,
            'short_desc': get_description_hh(vacancy)
        }
          
        # пополняем коллекцию без дубликатов
        obj = next(vacancies.find({"id": {'$eq': vacancy_data['id']}}), None)
        if not obj:
            vacancies.insert_one(vacancy_data)
        else:
            # обновляем объект в случае наличия в коллекции
            vacancies.update_one({"id": {'$eq': vacancy_data['id']}}, {'$set': vacancy_data})

### Superjob.ru parsing

In [10]:
# записываем классы
vac_block = '_1Ttd8 _2CsQi'
vac_class = '_3zucV f-test-vacancy-item _3j3cA RwN9e _3tNK- _1NStQ _1I1pc'        
title_url_class = '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'
salary_class = '_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'
employer_class = '_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'
location = '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'
desc_class = '_3mfro _3V-Qt _9fXTd _2JVkc _2VHxz'

In [11]:
def get_title_sj(vacancy):
    title = vacancy.findChildren('div', {'class': title_url_class})
    # обрабатываем случай, когда title лежит не в span, а в h2
    if title:
        return title[0].getText()
    else:
        return vacancy.findChildren('h2', {'class': title_url_class})[0].getText()


def get_employer_sj(vacancy):
    employer_el = vacancy.findChildren('span', {'class':employer_class})
    # обрабатываем случай, когда работодатель не указан
    if employer_el:
        return employer_el[0].getText()
    else:
        return None


def get_salary_sj(vacancy):
    salary_el = vacancy.findChildren('span', {'class': salary_class})[0].getText()
    min_salary, max_salary, currency = None, None, None

    if salary_el != 'По договорённости':
        #разбиваем по тире, заменяем неразрывные пробелы
        split_range = re.split('—', salary_el.replace('\xa0', ' '))

        # диапазон
        if len(split_range) == 2:
            min_salary = int(''.join(re.findall('\d+', split_range[0])))
            items = re.findall('\w+', split_range[1])
            max_salary = int(''.join(items[:-1]))
            currency = items[-1] 
        else:
            # разбиваем по пробелам список с одним элементом
            items_one = re.findall('\w+', split_range[0])
            # от или до
            if items_one[0] == 'от':
                min_salary = int(''.join(items_one[1:-1]))
            elif items_one[0] == 'до':
                max_salary = int(''.join(items_one[1:-1]) ) 
            # зп одним числом
            else:
                min_salary = int(''.join(items_one[:-1])) 
                max_salary = min_salary
            currency = items_one[-1]

    return min_salary, max_salary, currency


def get_url_sj(vacancy):
    url = vacancy.findChildren('div', {'class': title_url_class})
    # обрабатываем случай, когда url лежит не в span, а в h2
    if url:
        url = url[0].find('a')['href']
        return sj + url             
    else:
        url = vacancy.findChildren('h2', {'class': title_url_class})[0].find('a')['href']
        return sj + url
    
    
def get_id_sj(vacancy):
    return f"sj_{sj_id_pattern.findall(get_url_sj(vacancy))[0]}"


def get_description_sj(vacancy):
    desc_el = vacancy.findChildren('span', {'class': desc_class})
    # обрабатываем случай, когда описания вакансии нет
    if desc_el:
        return desc_el[0].getText()
    else:
        return None

In [12]:
for i in sj_page_count:
    # обрабатываем случай, когда регион 'москва'
    if sj_area == 'москва':
        sj_link = f'{sj}/vacancy/search/?keywords={query}&geo[t][0]=4&page={i}'
    else:
        sj_link = f'https://{sj_area}.superjob.ru/vacancy/search/?keywords={query}&page={i}'
    sj_page = requests.get(sj_link, headers=headers)

    if not sj_page.ok:
        break
        
    # формируем список элементов страницы, содержащих вакансии
    sj_soup = bs(sj_page.text,'lxml')
    vacancies_block = sj_soup.find_all('div', {'class': vac_block})[0]
    vacancies_list = vacancies_block.find_all('div', {'class': vac_class})
   
    # определяем условие выхода из цикла (вакансий больше нет)
    if len(vacancies_list) == 0:
        break

    # итерируемся по списку, содержащем вакансии
    for vacancy in vacancies_list:
        salary = get_salary_sj(vacancy)

        
        # записываем полученные значения в словарь по каждой вакансии
        vacancy_data = {
            'query': query,
            'area': area,
            'url': get_url_sj(vacancy),
            'id': get_id_sj(vacancy),
            'title': get_title_sj(vacancy),
            'employer': get_employer_sj(vacancy),
            'min_salary': salary[0],
            'max_salary': salary[1],
            'currency': salary[2],
            'source': sj,
            'short_desc': get_description_sj(vacancy)
        }


        # пополняем коллекцию без дубликатов
        obj = next(vacancies.find({"id": {'$eq': vacancy_data['id']}}), None)
        if not obj:
            vacancies.insert_one(vacancy_data)
        else:
            # обновляем объект в случае наличия в коллекции
            vacancies.update_one({"id": {'$eq': vacancy_data['id']}},{'$set': vacancy_data})

In [13]:
db.vacancies.count_documents({})

1370

In [14]:
df = pd.DataFrame(vacancies.find({}))
df

,_id,query,area,url,id,title,employer,min_salary,max_salary,currency,source,short_desc
0,5e973e8e59f84364d634f37e,data scientist,москва,https://kolomna.hh.ru/vacancy/36271196?query=d...,hh_36271196,NLP engineer / Data scientist NLP,ООО АлгоМост,100000.0,250000.0,руб,https://hh.ru/,"Кластеризация и классификация документов, извл..."
1,5e973e8e59f84364d634f37f,data scientist,москва,https://kolomna.hh.ru/vacancy/35895583?query=d...,hh_35895583,"Специалист / data scientist (big data, прогнос...",ЗАО Прогностические решения,100000.0,NaN,руб,https://hh.ru/,Участие в проектах по созданию и внедрению ана...
2,5e973e8e59f84364d634f380,data scientist,москва,https://kolomna.hh.ru/vacancy/36214189?query=d...,hh_36214189,Ведущий аналитик данных / Data Scientist,ООО ГЕТСИЭРЭМ,NaN,300000.0,руб,https://hh.ru/,Сбор требований и написание проектных решений....
3,5e973e8e59f84364d634f381,machine learning,москва,https://kolomna.hh.ru/vacancy/36626949?query=m...,hh_36626949,Data scientist (middle),Открытые Технологии,NaN,150000.0,руб,https://hh.ru/,Выявление аномалий во временных рядах и тексто...
4,5e973e8e59f84364d634f382,data scientist,москва,https://kolomna.hh.ru/vacancy/33481272?query=d...,hh_33481272,Senior Data scientist (NLP/NLU),ДомКлик,NaN,NaN,None,https://hh.ru/,Знакомы с классическими алгоритмами и структур...
...,...,...,...,...,...,...,...,...,...,...,...,...
1365,5e973f4bbb25f97da9ae1952,machine learning,москва,https://kolomna.hh.ru/vacancy/36209002?query=m...,hh_36209002,Системный администратор GNU / Linux,QSOFT,80000.0,100000.0,руб,https://hh.ru/,"Участие в развитии внутренней инфраструктуры, ..."
1366,5e973f4bbb25f97da9ae1953,machine learning,москва,https://kolomna.hh.ru/vacancy/36246794?query=m...,hh_36246794,Senior Backend Developer (Team Leader),ООО ЭНЛОЙТ,500000.0,500000.0,руб,https://hh.ru/,Working machine type by your request. Educatio...
1367,5e973f4bbb25f97da9ae1954,machine learning,москва,https://kolomna.hh.ru/vacancy/36254556?query=m...,hh_36254556,Back-end разработчик цифровой платформы,Счетная палата Российской Федерации,NaN,NaN,None,https://hh.ru/,Участие в создания Цифровой платформы Счетной ...
1368,5e973f4bbb25f97da9ae1955,machine learning,москва,https://kolomna.hh.ru/vacancy/36293188?query=m...,hh_36293188,Программист Python,GRISSLI,95000.0,NaN,руб,https://hh.ru/,Разработка агрегатора для сервиса продажи биле...


#### Реализация поиска и вывода на экран вакансий с заработной платой больше введенной суммы:

In [15]:
def get_salary_gt(salary_threshold):
    return pd.DataFrame(vacancies.find({
        '$or': [ 
            # Не указан максимум, но минимум больше salary_filter
            {'$and': [{'max_salary': None}, {'min_salary': {'$gt': salary_threshold}}]},
            # Максимум больше salary_filter
            {'max_salary': {'$gt': salary_threshold}}
        ]
    }))


get_salary_gt(int(input('Введите нижний порог заработной платы: ')))

Введите нижний порог заработной платы: 150000


,_id,query,area,url,id,title,employer,min_salary,max_salary,currency,source,short_desc
0,5e973e8e59f84364d634f37e,data scientist,москва,https://kolomna.hh.ru/vacancy/36271196?query=d...,hh_36271196,NLP engineer / Data scientist NLP,ООО АлгоМост,100000.0,250000.0,руб,https://hh.ru/,"Кластеризация и классификация документов, извл..."
1,5e973e8e59f84364d634f380,data scientist,москва,https://kolomna.hh.ru/vacancy/36214189?query=d...,hh_36214189,Ведущий аналитик данных / Data Scientist,ООО ГЕТСИЭРЭМ,NaN,300000.0,руб,https://hh.ru/,Сбор требований и написание проектных решений....
2,5e973e8e59f84364d634f383,data scientist,москва,https://kolomna.hh.ru/vacancy/36628688?query=d...,hh_36628688,Computer Vision Data scientist,"AROUND, Группа компаний",140000.0,180000.0,руб,https://hh.ru/,Поддержка и развитие нашего продукта: Интеграц...
3,5e973e8e59f84364d634f385,machine learning,москва,https://kolomna.hh.ru/vacancy/36359628?query=m...,hh_36359628,Senior Data Scientist,Gradient,300000.0,NaN,руб,https://hh.ru/,"...теория вероятностей, матанализ. Английский..."
4,5e973e8e59f84364d634f386,data scientist,москва,https://kolomna.hh.ru/vacancy/35601180?query=d...,hh_35601180,Data Scientist,ivi,170000.0,NaN,руб,https://hh.ru/,Работа с большими объемами данных. Подготовка ...
...,...,...,...,...,...,...,...,...,...,...,...,...
99,5e973f4abb25f97da9ae1905,machine learning,москва,https://kolomna.hh.ru/vacancy/36359528?query=m...,hh_36359528,Ведущий менеджер по продажам интернет-рекламы,ООО Гибрид,150000.0,500000.0,руб,https://hh.ru/,"Звонить, писать, встречаться и разговаривать с..."
100,5e973f4abb25f97da9ae190c,machine learning,москва,https://kolomna.hh.ru/vacancy/33727610?query=m...,hh_33727610,Senior / Lead QA,Центр финансовых технологий Базис,NaN,253000.0,руб,https://hh.ru/,Один из наших продуктов - собственный ипотечны...
101,5e973f4bbb25f97da9ae192a,machine learning,москва,https://kolomna.hh.ru/vacancy/36431622?query=m...,hh_36431622,Менеджер по продажам и развитию it-услуг,ООО Нэти,100000.0,300000.0,руб,https://hh.ru/,"Искать потенциальных клиентов, выходить на ЛПР..."
102,5e973f4bbb25f97da9ae1938,machine learning,москва,https://kolomna.hh.ru/vacancy/35943159?query=m...,hh_35943159,Senior developer C#,ООО Логистик Тайм,180000.0,230000.0,руб,https://hh.ru/,Разработка высоконагруженных систем. Поддержка...
